<a href="https://www.kaggle.com/code/gavrildsouza/airlines?scriptVersionId=107737206" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This R environment comes with many helpful analytics packages installed
# It is defined by the kaggle/rstats Docker image: https://github.com/kaggle/docker-rstats
# For example, here's a helpful package to load

#library(tidyverse) # metapackage of all tidyverse packages

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#list.files(path = "../input")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the data and basic understanding
**We load the data and get a basic visual and understanding about it**

In [2]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

In [3]:
#Importing Data
data = pd.read_csv('../input/airlines-dataset-to-predict-a-delay/Airlines.csv')
data.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [4]:
data.describe()

,id,Flight,DayOfWeek,Time,Length,Delay
count,539383.00000,539383.000000,539383.000000,539383.000000,539383.000000,539383.000000
mean,269692.00000,2427.928630,3.929668,802.728963,132.202007,0.445442
std,155706.60446,2067.429837,1.914664,278.045911,70.117016,0.497015
min,1.00000,1.000000,1.000000,10.000000,0.000000,0.000000
25%,134846.50000,712.000000,2.000000,565.000000,81.000000,0.000000
50%,269692.00000,1809.000000,4.000000,795.000000,115.000000,0.000000
75%,404537.50000,3745.000000,5.000000,1035.000000,162.000000,1.000000
max,539383.00000,7814.000000,7.000000,1439.000000,655.000000,1.000000


In [5]:
data.isnull().any()

id             False
Airline        False
Flight         False
AirportFrom    False
AirportTo      False
DayOfWeek      False
Time           False
Length         False
Delay          False
dtype: bool

In [6]:
data.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [7]:
data1 = data.copy()

In [8]:
# Preprocess the data
from sklearn import preprocessing

LE = preprocessing.LabelEncoder()
data1['Airline'] = LE.fit_transform(data1['Airline'])
data1['AirportFrom'] = LE.fit_transform(data1['AirportFrom'])
data1['AirportTo'] = LE.fit_transform(data1['AirportTo'])

In [9]:
data1.drop(columns=['id'], inplace=True)
data1.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,4,269,253,135,3,15,205,1
1,14,1558,217,60,3,15,222,1
2,1,2400,154,80,3,20,165,1
3,1,2466,253,80,3,20,195,1
4,2,108,14,252,3,30,202,0


# Feature Extraction
**We can now add more coloumns based on the existing data to make the dataset more usable**

In [10]:
# Categorizing Time as part of the day (Departure timing):
# Morning = 1, Afternoon = 2, Evening = 3, Night = 4
# ----------------------------------------------------
data1['TimeByHr'] = data1['Time']/60

DepartTime = []
for ti in range(0,len(data1)):
    if (data1['TimeByHr'][ti]>= 4) & (data1['TimeByHr'][ti]<=12):
        DepartTime.append(1)
    elif (data1['TimeByHr'][ti]>= 12) & (data1['TimeByHr'][ti]<=15):
        DepartTime.append(2)
    elif (data1['TimeByHr'][ti]>= 15) & (data1['TimeByHr'][ti]<=17):
        DepartTime.append(3)
    else:
        DepartTime.append(4)
        
data1['DepartTime'] = DepartTime

In [11]:
# Categorizing Day as Holiday or Not:
# Holiday = 1, NotHoliday = 0
# --------------------------------------

Holiday = []
for day in range(0,len(data1)):
    if (data1['DayOfWeek'][day]==6) | (data1['DayOfWeek'][day]==7):
        Holiday.append(1)
    else:
        Holiday.append(0)
        
data1['Holiday'] = Holiday

In [12]:
# : Determining the Flight duration 
# ----------------------------------------------------

data1['LenByHr'] = data1['Length']/60

AirTime = []
for Atime in range(0, len(data1)):
    if ((data1['LenByHr'][Atime] + data1['TimeByHr'][Atime])>=24):
        AirTime.append((data1['LenByHr'][Atime] + data1['TimeByHr'][Atime]) - 24)
    else:
        AirTime.append(data1['LenByHr'][Atime] + data1['TimeByHr'][Atime])
        
data1['Arrival'] = AirTime

In [13]:
data1.head()

,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay,TimeByHr,DepartTime,Holiday,LenByHr,Arrival
0,4,269,253,135,3,15,205,1,0.250000,4,0,3.416667,3.666667
1,14,1558,217,60,3,15,222,1,0.250000,4,0,3.700000,3.950000
2,1,2400,154,80,3,20,165,1,0.333333,4,0,2.750000,3.083333
3,1,2466,253,80,3,20,195,1,0.333333,4,0,3.250000,3.583333
4,2,108,14,252,3,30,202,0,0.500000,4,0,3.366667,3.866667


In [14]:
#We can drop the columns that are not needed by us
data1.drop(['Time','Length'], axis = 1, inplace = True)

In [15]:
#lets rearrange the columns to get a better visual
data1 = data1[['Airline', 'Flight', 'AirportFrom', 'AirportTo', 'Delay', 'DayOfWeek', 'Holiday', 'TimeByHr', 'LenByHr', 'DepartTime', 'Arrival']]
data1.head()

,Airline,Flight,AirportFrom,AirportTo,Delay,DayOfWeek,Holiday,TimeByHr,LenByHr,DepartTime,Arrival
0,4,269,253,135,1,3,0,0.250000,3.416667,4,3.666667
1,14,1558,217,60,1,3,0,0.250000,3.700000,4,3.950000
2,1,2400,154,80,1,3,0,0.333333,2.750000,4,3.083333
3,1,2466,253,80,1,3,0,0.333333,3.250000,4,3.583333
4,2,108,14,252,0,3,0,0.500000,3.366667,4,3.866667


# Visualization
**From here we can get a better visual and understanding of the data that we have**

In [16]:
import plotly.express as px
import plotly.graph_objs as pgo
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [17]:
data.head()

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
0,1,CO,269,SFO,IAH,3,15,205,1
1,2,US,1558,PHX,CLT,3,15,222,1
2,3,AA,2400,LAX,DFW,3,20,165,1
3,4,AA,2466,SFO,DFW,3,20,195,1
4,5,AS,108,ANC,SEA,3,30,202,0


In [18]:
#Adding Count col for easier plotting
data['Count'] = 1

In [19]:
# Bar chart showing most airbound flights
flight_count = data.groupby(['Airline'], as_index =False).Count.sum()
figure = px.bar(flight_count, x = 'Airline', y = 'Count')
figure.show()

In [20]:
# Showing which day the flights fly the most
most_flight = data.groupby(['DayOfWeek'], as_index = False).Count.sum()
fig = px.bar(most_flight, x = 'DayOfWeek', y = 'Count')
fig.show()

In [21]:
# Chart showing which airports do most flights fly to and on which day
flight_count = data.groupby(['DayOfWeek','AirportTo'], as_index =False).Count.sum()
figure = px.bar(flight_count, x = 'DayOfWeek', y = 'Count', color = 'AirportTo')
figure.show()

In [22]:
# Chart showing which airports do most flights fly to, sorted by the top 5
flight_count = data.groupby(['Airline','AirportTo'], as_index =False).Count.sum().sort_values('Count', ascending = False).head()
print(flight_count)
figure = px.bar(flight_count, x = 'Airline', y = 'Count', color = 'AirportTo')
figure.show()

     Airline AirportTo  Count
363       DL       ATL  15233
141       AA       DFW  12920
480       EV       ATL   9756
1343      XE       IAH   7934
1159      US       CLT   7090


In [23]:
# which part of the day do most of these flights go to the airports
# assign the day according to the departure time
data['Departure'] = DepartTime
maping = {1:'Morning', 2:'Afternoon', 3:'Evening', 4:'Night'}
data.replace({'Departure':maping}, inplace = True)

Departure_count = data.groupby(['Airline','Departure'], as_index =False).Count.sum()
figure = px.bar(Departure_count, x = 'Departure', y = 'Count', color = 'Airline')
figure.show()

# Relations between variables
**using plotly we will plot a heatmap to show the corelation between the variable**

In [24]:
corr = data1.corr()
HeatFig = px.imshow(corr,
                    x = list(corr.columns), y = list(corr.columns),
                    width = 1000, height = 1000,)
HeatFig.show()